# Age of mitologhy: Tournament match

**Project**:       Age of mitologhy tournament   
**Author**:        Hugo Fernandez  
**Output**:        Dataframe with Teams  
**Description**:   This code build teams of 2 participants balanced by experience in the game 

## Importing data from google sheet 
The data was storage in my personal google drive and feeded by a google form.  


In [1]:
import numpy as np
from tabulate import tabulate
import pandas as pd
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow,Flow
from google.auth.transport.requests import Request
import os
import pickle

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# here enter the id of your google sheet
SAMPLE_SPREADSHEET_ID_input = '1C9oYlZq1GVXnxu3AJ-Y6XIyyR8enZbxn4BwwcAN4lbM'
SAMPLE_RANGE_NAME = 'A1:AA1000'

def main():
    global values_input, service
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES) # here enter the name of your downloaded JSON file
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result_input = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID_input,
                                range=SAMPLE_RANGE_NAME).execute()
    values_input = result_input.get('values', [])

    if not values_input and not values_expansion:
        print('No data found.')

main()

participants=pd.DataFrame(values_input[1:], columns=values_input[0])

## Exploratory analysis

In [2]:
participants.columns.map(type)

Index([<class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>,
       <class 'str'>, <class 'str'>, <class 'str'>],
      dtype='object')

In [3]:
participants.count()

Marca temporal                                                                                  16
Cuál es tu nickname de steam?                                                                   16
Cuál es tu verdadero nombre?                                                                    16
Cuál es tu nivel en el juego (real rate)? Sean sinceros a los dioses no les gusta la mentira    16
Exactamente cuál crees que sea tu real rate?                                                    16
Cómo te contactamos?                                                                            16
Tu usuario o número para contactarte con tus compañeros y contrincantes                         16
dtype: int64

In [4]:
participants.head(16)

,Marca temporal,Cuál es tu nickname de steam?,Cuál es tu verdadero nombre?,Cuál es tu nivel en el juego (real rate)? Sean sinceros a los dioses no les gusta la mentira,Exactamente cuál crees que sea tu real rate?,Cómo te contactamos?,Tu usuario o número para contactarte con tus compañeros y contrincantes
0,27/03/2021 11:14:58,hfq,Hugo Fernandez,Noob (0 - 1699),1580,Telegram,@xHugoFQx
1,27/03/2021 12:34:10,WeroPainkiller,Adrian Becerra,Noob + (1700 - 1799),1750,Telegram,Adrian Becerra
2,27/03/2021 12:36:42,jmedinae,Sebastián Medina,Noob + (1700 - 1799),1725,Telegram,+57 3204055441
3,27/03/2021 12:49:49,Comm_V8,Isaac,Noob ++ (1800 - 1899),1850 en Steam.,Telegram,Yogel/Eihen/Isaac
4,27/03/2021 12:52:45,juandelacruz23,Juan De la Cruz,Noob ++ (1800 - 1899),1850,Telegram,9932968952
5,27/03/2021 13:49:27,Torn,Leandro,Noob (0 - 1699),1700,Telegram,LeaNoBella
6,28/03/2021 10:28:19,AegisArc,Alexis,Noob (0 - 1699),1400,Telegram,AegisArc
7,28/03/2021 10:32:27,atenea21,Tania :),Noob (0 - 1699),Pense que 1650 pero estuve jugando partidas rá...,Telegram,Mi usuario de steam? atenea21
8,28/03/2021 10:41:40,filosoraptor,Renzo Castillo,Noob (0 - 1699),Creo que aún soy noob,Telegram,filosoraptor21
9,28/03/2021 19:30:53,Niisan,Ricardo,Noob ++ (1800 - 1899),"de voobly 1650, steam 1850",Discord,-Dex-#0512


In [5]:
participants = participants.rename(columns={'Marca temporal': 'timestamp', 
                             'Cuál es tu nickname de steam?': 'id_steam',
                            'Cuál es tu verdadero nombre?':'name',
                            'Cuál es tu nivel en el juego (real rate)? Sean sinceros a los dioses no les gusta la mentira':'level_player',
                            'Exactamente cuál crees que sea tu real rate?':'real_rate',
                             'Cómo te contactamos?':'messenger_service', 
                             'Tu usuario o número para contactarte con tus compañeros y contrincantes':'messenger_service_id'})

In [6]:
#Cleaning real rate
participants['real_rate'] = participants['real_rate'].replace(['Creo que aún soy noob','Ni idea'],'1600')
participants['real_rate'] = participants['real_rate'].replace(['de voobly 1650, steam 1850','1850 en Steam. '],'1850')
participants['real_rate'] = participants['real_rate'].replace('Pense que 1650 pero estuve jugando partidas rápidas y baje el rate. Diría 1620 jaja','1635')
participants['real_rate'] = participants['real_rate'].replace('+1900','1900')
participants['real_rate'] = participants['real_rate'].astype(int) 

#Update steam ID
participants['id_steam'] = participants['id_steam'].replace('cristobalvgb','Cris scarmander')

In [7]:
participants.head(16)

,timestamp,id_steam,name,level_player,real_rate,messenger_service,messenger_service_id
0,27/03/2021 11:14:58,hfq,Hugo Fernandez,Noob (0 - 1699),1580,Telegram,@xHugoFQx
1,27/03/2021 12:34:10,WeroPainkiller,Adrian Becerra,Noob + (1700 - 1799),1750,Telegram,Adrian Becerra
2,27/03/2021 12:36:42,jmedinae,Sebastián Medina,Noob + (1700 - 1799),1725,Telegram,+57 3204055441
3,27/03/2021 12:49:49,Comm_V8,Isaac,Noob ++ (1800 - 1899),1850,Telegram,Yogel/Eihen/Isaac
4,27/03/2021 12:52:45,juandelacruz23,Juan De la Cruz,Noob ++ (1800 - 1899),1850,Telegram,9932968952
5,27/03/2021 13:49:27,Torn,Leandro,Noob (0 - 1699),1700,Telegram,LeaNoBella
6,28/03/2021 10:28:19,AegisArc,Alexis,Noob (0 - 1699),1400,Telegram,AegisArc
7,28/03/2021 10:32:27,atenea21,Tania :),Noob (0 - 1699),1635,Telegram,Mi usuario de steam? atenea21
8,28/03/2021 10:41:40,filosoraptor,Renzo Castillo,Noob (0 - 1699),1600,Telegram,filosoraptor21
9,28/03/2021 19:30:53,Niisan,Ricardo,Noob ++ (1800 - 1899),1850,Discord,-Dex-#0512


In [8]:
participants.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
real_rate,16.0,1703.75,131.510456,1400.0,1600.0,1710.0,1812.5,1900.0


## Matching

In [9]:
#Sort participants
participants = participants.sort_values('real_rate',kind='mergesort')
participants['id_rank'] = range(len(participants))

#Split participants
lower_participants = participants.iloc[:8,:]
higther_participants = participants.iloc[8:16,:]

#Match by id_rank
higther_participants = higther_participants.sort_values('real_rate', ascending = False,kind='mergesort' )
higther_participants['id_rank'] = range(len(higther_participants))
inner_join = lower_participants.merge(higther_participants, on=['id_rank'], how="inner")
inner_join['mean_rr_team'] =  (inner_join['real_rate_x'] + inner_join['real_rate_y'])/2


In [19]:
inner_join[['id_steam_x','id_steam_y','mean_rr_team']].head(8)

,id_steam_x,id_steam_y,mean_rr_team
0,AegisArc,ZR1-Atrx,1650.0
1,hfq,Comm_V8,1715.0
2,filosoraptor,juandelacruz23,1725.0
3,Cris scarmander,Niisan,1725.0
4,JZG,Angeltrilli,1700.0
5,atenea21,WeroPainkiller,1692.5
6,Torn,jmedinae,1712.5
7,Vizan,eduardo1302,1710.0
